# Sessie 5 – Multi-Agent Orchestrator

Toont een eenvoudige tweestaps-pijplijn (Onderzoeker -> Redacteur) met behulp van Foundry Local.


### Uitleg: Installatie van afhankelijkheden
Installeert `foundry-local-sdk` en `openai` die nodig zijn voor lokale modeltoegang en chatvoltooiingen. Idempotent.


# Scenario
Implementeert een minimaal twee-agent orkestratiepatroon:
- **Onderzoeker-agent** verzamelt beknopte feitelijke punten
- **Redacteur-agent** herschrijft voor duidelijke executive communicatie

Toont gedeeld geheugen per agent, sequentiële overdracht van tussentijdse output en een eenvoudige pipelinefunctie. Uitbreidbaar naar meer rollen (bijv. Criticus, Verificateur) of parallelle takken.

**Omgevingsvariabelen:**
- `FOUNDRY_LOCAL_ALIAS` - Standaardmodel om te gebruiken (standaard: phi-4-mini)
- `AGENT_MODEL_PRIMARY` - Primair agentmodel (overschrijft ALIAS)
- `AGENT_MODEL_EDITOR` - Redacteur-agentmodel (standaard: primair model)

**SDK Referentie:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local

**Hoe het werkt:**
1. **FoundryLocalManager** start automatisch de Foundry Local-service
2. Downloadt en laadt het opgegeven model (of gebruikt de gecachte versie)
3. Biedt een OpenAI-compatibel eindpunt voor interactie
4. Elke agent kan een ander model gebruiken voor gespecialiseerde taken
5. Ingebouwde retry-logica behandelt tijdelijke fouten op een elegante manier

**Belangrijkste kenmerken:**
- ✅ Automatische serviceontdekking en initialisatie
- ✅ Modellevenscyclusbeheer (downloaden, cachen, laden)
- ✅ OpenAI SDK-compatibiliteit voor een vertrouwde API
- ✅ Ondersteuning voor meerdere modellen voor agentspecialisatie
- ✅ Robuuste foutafhandeling met retry-logica
- ✅ Lokale inferentie (geen cloud-API vereist)


In [16]:
# Install dependencies
!pip install -q foundry-local-sdk openai

### Uitleg: Kernimporten & Typing
Introduceert dataclasses voor het opslaan van agentberichten en typing hints voor duidelijkheid. Importeert Foundry Local manager + OpenAI client voor daaropvolgende acties van de agent.


In [17]:
from dataclasses import dataclass, field
from typing import List
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI

### Uitleg: Model Initialisatie (SDK-patroon)
Maakt gebruik van de Foundry Local Python SDK voor robuust modelbeheer:
- **FoundryLocalManager(alias)** - Start automatisch de service en laadt het model via alias
- **get_model_info(alias)** - Verbindt alias met een concreet model-ID
- **manager.endpoint** - Biedt de service-endpoint voor de OpenAI-client
- **manager.api_key** - Biedt een API-sleutel (optioneel voor lokaal gebruik)
- Ondersteunt aparte modellen voor verschillende agents (primair vs. editor)
- Ingebouwde retry-logica met exponentiële backoff voor veerkracht
- Verificatie van de verbinding om te garanderen dat de service klaar is

**Belangrijk SDK-patroon:**
```python
manager = FoundryLocalManager(alias)
model_info = manager.get_model_info(alias)
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key)
```

**Levenscyclusbeheer:**
- Managers worden globaal opgeslagen voor correcte opruiming
- Elke agent kan een ander model gebruiken voor specialisatie
- Automatische service-detectie en verbindingafhandeling
- Soepele retry met exponentiële backoff bij fouten

Dit zorgt voor een correcte initialisatie voordat de orkestratie van agents begint.

**Referentie:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local


In [18]:
import time

# Environment configuration
PRIMARY_ALIAS = os.getenv('AGENT_MODEL_PRIMARY', os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini'))
EDITOR_ALIAS = os.getenv('AGENT_MODEL_EDITOR', PRIMARY_ALIAS)

# Store managers globally for proper lifecycle management
primary_manager = None
editor_manager = None

def init_model(alias: str, max_retries: int = 3):
    """Initialize Foundry Local manager with retry logic.
    
    Args:
        alias: Model alias to initialize
        max_retries: Number of retry attempts with exponential backoff
    
    Returns:
        Tuple of (manager, client, model_id, endpoint)
    """
    delay = 2.0
    last_err = None
    
    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Init] Starting Foundry Local for '{alias}' (attempt {attempt}/{max_retries})...")
            
            # Initialize manager - this starts the service and loads the model
            manager = FoundryLocalManager(alias)
            
            # Get model info to retrieve the actual model ID
            model_info = manager.get_model_info(alias)
            model_id = model_info.id
            
            # Create OpenAI client with manager's endpoint
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key or 'not-needed'
            )
            
            # Verify the connection with a simple test
            models = client.models.list()
            print(f"[OK] Initialized '{alias}' -> {model_id} at {manager.endpoint}")
            
            return manager, client, model_id, manager.endpoint
            
        except Exception as e:
            last_err = e
            if attempt < max_retries:
                print(f"[Retry {attempt}/{max_retries}] Failed to init '{alias}': {e}")
                print(f"[Retry] Waiting {delay:.1f}s before retry...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[ERROR] Failed to initialize '{alias}' after {max_retries} attempts")
    
    raise RuntimeError(f"Failed to initialize '{alias}' after {max_retries} attempts: {last_err}")

# Initialize primary model (for researcher)
print(f"\n{'='*80}")
print(f"Initializing Primary Model: {PRIMARY_ALIAS}")
print('='*80)
primary_manager, primary_client, PRIMARY_MODEL_ID, primary_endpoint = init_model(PRIMARY_ALIAS)

# Initialize editor model (may be same as primary)
if EDITOR_ALIAS != PRIMARY_ALIAS:
    print(f"\n{'='*80}")
    print(f"Initializing Editor Model: {EDITOR_ALIAS}")
    print('='*80)
    editor_manager, editor_client, EDITOR_MODEL_ID, editor_endpoint = init_model(EDITOR_ALIAS)
else:
    print(f"\n[Info] Editor using same model as primary")
    editor_manager = primary_manager
    editor_client, EDITOR_MODEL_ID = primary_client, PRIMARY_MODEL_ID
    editor_endpoint = primary_endpoint

print(f"\n{'='*80}")
print(f"[Configuration Summary]")
print('='*80)
print(f"  Primary Agent:")
print(f"    - Alias: {PRIMARY_ALIAS}")
print(f"    - Model: {PRIMARY_MODEL_ID}")
print(f"    - Endpoint: {primary_endpoint}")
print(f"\n  Editor Agent:")
print(f"    - Alias: {EDITOR_ALIAS}")
print(f"    - Model: {EDITOR_MODEL_ID}")
print(f"    - Endpoint: {editor_endpoint}")
print('='*80)



Initializing Primary Model: phi-4-mini
[Init] Starting Foundry Local for 'phi-4-mini' (attempt 1/3)...
[OK] Initialized 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4 at http://127.0.0.1:59959/v1

Initializing Editor Model: gpt-oss-20b
[Init] Starting Foundry Local for 'gpt-oss-20b' (attempt 1/3)...
[OK] Initialized 'gpt-oss-20b' -> gpt-oss-20b-cuda-gpu:1 at http://127.0.0.1:59959/v1

[Configuration Summary]
  Primary Agent:
    - Alias: phi-4-mini
    - Model: Phi-4-mini-instruct-cuda-gpu:4
    - Endpoint: http://127.0.0.1:59959/v1

  Editor Agent:
    - Alias: gpt-oss-20b
    - Model: gpt-oss-20b-cuda-gpu:1
    - Endpoint: http://127.0.0.1:59959/v1


### Uitleg: Agent- en Memory-klassen
Definieert de eenvoudige `AgentMsg` voor geheugenitems en `Agent` die het volgende omvat:
- **Systeemrol** - De persona en instructies van de agent
- **Berichtgeschiedenis** - Behoudt de context van het gesprek
- **act() methode** - Voert acties uit met correcte foutafhandeling

De agent kan verschillende modellen gebruiken (primair versus editor) en behoudt een geïsoleerde context per agent. Dit patroon maakt het mogelijk:
- Geheugenbehoud tussen acties
- Flexibele modeltoewijzing per agent
- Foutisolatie en herstel
- Eenvoudig koppelen en orkestreren


In [19]:
@dataclass
class AgentMsg:
    role: str
    content: str

@dataclass
class Agent:
    name: str
    system: str
    client: OpenAI = None  # Allow per-agent client assignment
    model_id: str = None   # Allow per-agent model
    memory: List[AgentMsg] = field(default_factory=list)

    def _history(self):
        """Return chat history in OpenAI messages format including system + memory."""
        msgs = [{'role': 'system', 'content': self.system}]
        for m in self.memory[-6:]:  # Keep last 6 messages to avoid context overflow
            msgs.append({'role': m.role, 'content': m.content})
        return msgs

    def act(self, prompt: str, temperature: float = 0.4, max_tokens: int = 300):
        """Send a prompt, store user + assistant messages in memory, and return assistant text.
        
        Args:
            prompt: User input/task for the agent
            temperature: Sampling temperature (0.0-1.0)
            max_tokens: Maximum tokens to generate
        
        Returns:
            Assistant response text
        """
        # Use agent-specific client/model or fall back to primary
        client_to_use = self.client or primary_client
        model_to_use = self.model_id or PRIMARY_MODEL_ID
        
        self.memory.append(AgentMsg('user', prompt))
        
        try:
            # Build messages including system prompt and history
            messages = self._history() + [{'role': 'user', 'content': prompt}]
            
            resp = client_to_use.chat.completions.create(
                model=model_to_use,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            
            # Validate response
            if not resp.choices:
                raise RuntimeError("No completion choices returned")
            
            out = resp.choices[0].message.content or ""
            
            if not out:
                raise RuntimeError("Empty response content")
            
        except Exception as e:
            out = f"[ERROR:{self.name}] {type(e).__name__}: {str(e)}"
            print(f"[Agent Error] {self.name}: {type(e).__name__}: {str(e)}")
        
        self.memory.append(AgentMsg('assistant', out))
        return out

print("[INFO] Agent classes initialized with Foundry SDK support")
print(f"[INFO] Using OpenAI SDK version: {OpenAI.__module__}")


[INFO] Agent classes initialized with Foundry SDK support
[INFO] Using OpenAI SDK version: openai


### Uitleg: Georkestreerde Pijplijn
Creëert twee gespecialiseerde agenten:
- **Onderzoeker**: Gebruikt het primaire model, verzamelt feitelijke informatie
- **Redacteur**: Kan een apart model gebruiken (indien geconfigureerd), verfijnt en herschrijft

De `pipeline`-functie:
1. Onderzoeker verzamelt ruwe informatie
2. Redacteur verfijnt dit tot uitvoeringsklare output
3. Geeft zowel tussentijdse als eindresultaten terug

Dit patroon maakt het mogelijk:
- Modelspecialisatie (verschillende modellen voor verschillende rollen)
- Kwaliteitsverbetering door meerfasige verwerking
- Traceerbaarheid van informatie-transformatie
- Eenvoudige uitbreiding naar meer agenten of parallelle verwerking


In [ ]:
# Create specialized agents with optional model assignment
researcher = Agent(
    name='Researcher',
    system='You collect concise factual bullet points.',
    client=primary_client,
    model_id=PRIMARY_MODEL_ID
)

editor = Agent(
    name='Editor',
    system='You rewrite content for clarity and an executive, action-focused tone.',
    client=editor_client,
    model_id=EDITOR_MODEL_ID
)

def pipeline(q: str, verbose: bool = True):
    """Execute multi-agent pipeline: Researcher -> Editor.
    
    Args:
        q: User question/task
        verbose: Print intermediate outputs
    
    Returns:
        Dictionary with research, final outputs, and metadata
    """
    if verbose:
        print(f"[Pipeline] Question: {q}\n")
    
    # Stage 1: Research
    if verbose:
        print("[Stage 1: Research]")
    research = researcher.act(q)
    if verbose:
        print(f"Output: {research[:200]}...\n")
    
    # Stage 2: Editorial refinement
    if verbose:
        print("[Stage 2: Editorial Refinement]")
    rewrite = editor.act(
        f"Rewrite professionally with a 1-sentence executive summary first. "
        f"Improve clarity, keep bullet structure if present. Source:\n{research}"
    )
    if verbose:
        print(f"Output: {rewrite[:200]}...\n")
    
    return {
        'question': q,
        'research': research,
        'final': rewrite,
        'models': {
            'researcher': PRIMARY_MODEL_ID,
            'editor': EDITOR_MODEL_ID
        }
    }

# Execute sample pipeline
print("="*80)
result = pipeline('Explain why edge AI matters for compliance and latency.')
print("="*80)
print("\n[FINAL OUTPUT]")
print(result['final'])
print("\n[METADATA]")
print(f"Models used: {result['models']}")
result

[Pipeline] Question: Explain why edge AI matters for compliance and latency.

[Stage 1: Research]
Output: - **Data Sovereignty**: Edge AI allows data to be processed locally, which can help organizations comply with regional data protection regulations by keeping sensitive information within the borders o...

[Stage 2: Editorial Refinement]


### Uitleg: Uitvoering van de Pipeline & Resultaten
Voert de multi-agent pipeline uit op een vraag met een compliance- en latency-thema om te demonstreren:
- Meerstaps informatie-transformatie
- Specialisatie en samenwerking tussen agents
- Verbetering van de outputkwaliteit door verfijning
- Traceerbaarheid (zowel tussentijdse als definitieve outputs worden behouden)

**Resultaatstructuur:**
- `question` - Originele gebruikersvraag
- `research` - Ruwe onderzoeksoutput (feitelijke punten)
- `final` - Verfijnde samenvatting voor executives
- `models` - Welke modellen zijn gebruikt in elke fase

**Uitbreidingsideeën:**
1. Voeg een Critic-agent toe voor kwaliteitscontrole
2. Implementeer parallelle onderzoeksagents voor verschillende aspecten
3. Voeg een Verifier-agent toe voor fact-checking
4. Gebruik verschillende modellen voor verschillende complexiteitsniveaus
5. Implementeer feedbackloops voor iteratieve verbetering


### Geavanceerd: Aangepaste Agentconfiguratie

Probeer het gedrag van de agent aan te passen door omgevingsvariabelen te wijzigen voordat je de initialisatiecel uitvoert:

**Beschikbare Modellen:**
- Gebruik `foundry model ls` in de terminal om alle beschikbare modellen te bekijken
- Voorbeelden: phi-4-mini, phi-3.5-mini, qwen2.5-7b, llama-3.2-3b, enzovoort.


In [ ]:
# Example: Use different models for different agents
# Uncomment and modify as needed:

# import os
# os.environ['AGENT_MODEL_PRIMARY'] = 'phi-4-mini'      # Fast, good for research
# os.environ['AGENT_MODEL_EDITOR'] = 'qwen2.5-7b'       # Higher quality for editing

# Then restart the kernel and re-run all cells

# Test with different questions
test_questions = [
    "What are 3 key benefits of using small language models?",
    "How does RAG improve AI accuracy?",
    "Why is local inference important for privacy?"
]

print("Testing pipeline with multiple questions:\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {q}")
    print('='*80)
    r = pipeline(q, verbose=False)
    print(f"\n[FINAL]: {r['final'][:300]}...")
    print(f"[Models]: Researcher={r['models']['researcher']}, Editor={r['models']['editor']}")


Testing pipeline with multiple questions:


Question 1: What are 3 key benefits of using small language models?

[FINAL]: <|channel|>analysis<|message|>The user wants a rewrite of the entire block of text. The rewrite should be professional, include a one-sentence executive summary first, improve clarity, keep bullet structure if present. The user has provided a large amount of text. The user wants a rewrite of that te...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 2: How does RAG improve AI accuracy?

[FINAL]: <|channel|>final<|message|>**RAG (Retrieval‑Augmented Generation) empowers AI to produce highly accurate, contextually relevant responses by combining a retrieval system with a large language model (LLM).**<|return|>...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 3: Why is local inference important for privacy?

[FINAL]: <|channel|>final<|message|>**Local inference—processing data d


---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in de oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor cruciale informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
